# 과제

#### 본인이 관심 있는 키워드에 대해서 해당 크롤링을 반복해서 수행하시오. 데이터를 수집하여 excel 파일로 저장하시오.

In [26]:
!pip install selenium

In [27]:
pip install chromedriver-autoinstaller

Note: you may need to restart the kernel to use updated packages.


In [28]:
#1. 필요한 라이브러리 import 하기 (10점)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

driver = webdriver.Chrome()

driver.get('https://www.gmarket.co.kr')

search_box = driver.find_element(By.NAME, 'keyword')

search_box.send_keys('스탠드')
search_box.send_keys(Keys.RETURN)

driver.quit()

In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

title_list = list()

# ChromeDriver의 경로를 지정하고, WebDriver 인스턴스를 생성합니다.
driver = webdriver.Chrome()

# 지마켓의 베스트 페이지를 엽니다.
driver.get('https://www.gmarket.co.kr/n/best')

# 상품명을 입력 받아 item_list 변수에 저장합니다.
item_list = driver.find_elements(By.CSS_SELECTOR, 'a.itemname')

for item in item_list:
    title_list.append(item.text)

driver.quit()

In [30]:
#2. 본인이 관심 있는 키워드에 대해서 크롤링을 수행하고 상품명, 가격, 별점 정보 저장하기 (40점)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

title_list = list()
price_list = list()
star_list = list()

# ChromeDriver의 경로를 지정하고, WebDriver 인스턴스를 생성합니다.
driver = webdriver.Chrome()

# 지마켓의 메인 페이지를 엽니다.
driver.get('https://www.gmarket.co.kr')

# 검색어를 입력할 수 있는 input 요소를 찾습니다.
search_box = driver.find_element(By.NAME, 'keyword')

# 검색어를 입력하고, Enter 키를 눌러 검색을 실행합니다.
search_box.send_keys('스탠드')
search_box.send_keys(Keys.RETURN)

# 결과 페이지가 로드될 때까지 잠시 기다립니다.
driver.implicitly_wait(3)

# 정보를 담은 박스를 선택합니다.
search_results = driver.find_elements(By.CSS_SELECTOR, 'div.box__information')

for result in search_results:
    #만족도가 없을 수도 있기에 try except문을 활용
    try:
        title_element = result.find_element(By.CSS_SELECTOR, 'span.text__item')
        price_element = result.find_element(By.CSS_SELECTOR, 'div.box__price-seller > strong')
        star_element = result.find_element(By.CSS_SELECTOR, 'span.image__awards-points > span')
        title_list.append(title_element.text)
        price_list.append(price_element.text)
        star_list.append(star_element.text)
    except:
        pass
    
# 작업이 끝나면 WebDriver를 종료하여 브라우저 창을 닫습니다.
driver.quit()

In [31]:
#3. 적절히 데이터 전처리를 하여 DataFrame 객체로 만들고 Excel 파일로 저장하기 (50점)

#이때, DataFrame의 열은 "상품명, 판매가, 만족도"로 구성되어야 함.
import pandas as pd

gmarket_selenium_df = pd.DataFrame([title_list, price_list, star_list]).T

gmarket_selenium_df.columns = ['상품명', '판매가', '만족도']

gmarket_selenium_df

,상품명,판매가,만족도
0,LED 스탠드 무드등 독서등 책상 조명 스텐드 3단 유선,"18,920",
1,액센트리 LED 스탠드 학습용 책상 독서등 시력보호 램프 AL-05S01(무선-휴대...,"27,900",
2,NEXT-120LAMP휴대용충전식 시력보호 3단 LED스탠드 누구나 쉽게 사용 가능,"12,600",
3,블루라이트 차단 LED 무선 스탠드 WSTD-1000 실버 스마일배송,"39,900",
4,테드 LED 와이드스탠드 SL-W800D 데스크형 화이트,"65,900",만족도 94% 입니다.
...,...,...,...
92,DP-910LS LED 학습용 공부 책상 조명 스탠드 스텐드,"43,800",만족도 96% 입니다.
93,LED와이드스탠드 조명 800B (화이트색상) 스탠드타입,"60,000",만족도 90% 입니다.
94,LED 거실 스탠드조명 플로어 장스탠드 무드등 간접조명 밝기조절 무선리모컨,"59,800",만족도 96% 입니다.
95,감성 캠핑 차박 낚시 조명 테이블 랜턴 스탠드 거치대,"13,800",만족도 86% 입니다.


In [32]:
def extract_comma(x):
    price = int(x.replace(",","")) #replace로 쉼표까지 제거하고 int로 변환
    return price

gmarket_selenium_df['판매가'] = gmarket_selenium_df['판매가'].apply(extract_comma)

import re

def extract_stars(x):
    ext = re.findall("\d+%", x)
    #데이터가 빈 경우가 있으므로 try, except문 사용
    try:
        stars = int(ext[0].replace('%', "")) #%를 제거한 뒤 int로 변환
    except:
        pass
    return stars if ext else None #데이터가 있을 경우 stars를 반환, 아닐 경우 None을 반환

gmarket_selenium_df['만족도'] = gmarket_selenium_df['만족도'].apply(extract_stars)

gmarket_selenium_df = gmarket_selenium_df.dropna().reset_index().iloc[:, 1:]

gmarket_selenium_df.to_excel('gmarket_stand_stars.xlsx')